<a href="https://colab.research.google.com/github/simonprudhomme/optimization_with_ortools/blob/main/or_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automated Reshuffling 

In [39]:
# install ortools
!pip install ortools

In [67]:
# import packages
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import numpy as np
import json

In [68]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver('factory', problem_type=pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [69]:
demand_per_fsa = {'H2H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H3H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H4H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H5H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H6H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H7H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H8H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,50),
                          'distance_station2': np.random.randint(0,50)
                          },
                  'H9H': {'parcels' : np.random.randint(0,1000),
                          'distance_station1': np.random.randint(0,100),
                          'distance_station2': np.random.randint(0,100)
                          }
                  }

print(json.dumps(demand_per_fsa, indent = 4))

{
    "H2H": {
        "parcels": 450,
        "distance_station1": 49,
        "distance_station2": 3
    },
    "H3H": {
        "parcels": 497,
        "distance_station1": 32,
        "distance_station2": 27
    },
    "H4H": {
        "parcels": 150,
        "distance_station1": 32,
        "distance_station2": 41
    },
    "H5H": {
        "parcels": 27,
        "distance_station1": 7,
        "distance_station2": 30
    },
    "H6H": {
        "parcels": 677,
        "distance_station1": 49,
        "distance_station2": 47
    },
    "H7H": {
        "parcels": 973,
        "distance_station1": 0,
        "distance_station2": 45
    },
    "H8H": {
        "parcels": 693,
        "distance_station1": 13,
        "distance_station2": 32
    },
    "H9H": {
        "parcels": 31,
        "distance_station1": 43,
        "distance_station2": 7
    }
}


In [71]:
# Create the variables

options = {}
for fsa in demand_per_fsa.keys():
  option_tmp = {}
  for station in stations:
    option_tmp[f'{fsa}_{station}'] = solver.IntVar(0, 1, f'{fsa}_{station}')
  options[f'{fsa}']=option_tmp

solver.NumVariables()

16

In [72]:
# Create the constraints

for fsa in options.keys():
  solver.Add(options[fsa][f'{fsa}_station_1'] + options[fsa][f'{fsa}_station_2'] == 1)

solver.Add(solver.Sum([(options[fsa][f'{fsa}_station_1'] * demand_per_fsa[fsa]['parcels'] + options[fsa][f'{fsa}_station_2'] * demand_per_fsa[fsa]['parcels']) for fsa in demand_per_fsa.keys()])==sum([demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()]))

solver.Add(solver.Sum([options[fsa][f'{fsa}_station_1'] * demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()]) <= 2000)
solver.Add(solver.Sum([options[fsa][f'{fsa}_station_2'] * demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()]) >= 2000)
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 11


In [73]:
# Create the objective function : reduce distance from FSA to STATIONS while respecting stations capacity

solver.Minimize(solver.Sum([options[fsa][f'{fsa}_station_1'] * demand_per_fsa[fsa]['distance_station1'] +  options[fsa][f'{fsa}_station_2'] * demand_per_fsa[fsa]['distance_station2'] for fsa in demand_per_fsa.keys()]))

In [74]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    station_1 = 0
    station_2 = 0
    for fsa in demand_per_fsa.keys():
      if options[fsa][f'{fsa}_station_1'].solution_value() >= 1.:
        print(f'{fsa} allocated to station 1')
        station_1 += demand_per_fsa[fsa]['parcels']
      else:
        print(f'{fsa} allocated to station 2')
        station_2 += demand_per_fsa[fsa]['parcels']
    print('Total parcels station1:', station_1)
    print('Total parcels station2:', station_2)

else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 155.0
H2H allocated to station 2
H3H allocated to station 2
H4H allocated to station 1
H5H allocated to station 1
H6H allocated to station 2
H7H allocated to station 1
H8H allocated to station 2
H9H allocated to station 2
Total parcels station1: 1150
Total parcels station2: 2348


In [66]:
sum([demand_per_fsa[fsa]['parcels'] for fsa in demand_per_fsa.keys()])

3038